<a href="https://colab.research.google.com/github/nnarey/NBA-MVP-Predictor/blob/main/Testing_the_model_on_years_with_new_models_with_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("2008-2023.csv")
df

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'AST')","('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share
0,0,0,Carmelo Anthony,SF,23,DEN,77,2806,21.1,0.568,...,3.4,1.3,0.5,0.492,0.354,0.786,8.2,0.140,2008,NaN
1,1,1,Gilbert Arenas,PG,26,WAS,13,425,18.2,0.529,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,NaN
2,2,2,Kobe Bryant,SG,29,LAL,82,3192,24.2,0.576,...,5.4,1.8,0.5,0.459,0.361,0.840,13.8,0.208,2008,NaN
3,3,3,Caron Butler,SF,27,WAS,58,2314,20.7,0.558,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,NaN
4,4,4,Vince Carter,SG,31,NJN,76,2959,18.8,0.550,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,48,48,Fred VanVleet,PG,28,TOR,69,2535,17.0,0.540,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0
508,49,49,Devin Vassell,SG,22,SAS,38,1178,15.5,0.550,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0
509,50,50,Franz Wagner,SF,21,ORL,80,2609,15.9,0.589,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0
510,51,51,Russell Westbrook,PG,34,TOT,73,2126,16.1,0.513,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.0


## 2023 Test - Jokic Predicted MVP, Embiid Actual

In [ ]:
df_2023 = df[df["Season"] == 2023].fillna(0)
df_no_2023 = df[df["Season"] != 2023].fillna(0)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

# Changed shooting percentages to TS% as it is more effective and added advanced stats: Player Efficiency Rating, Box Plus Minus,
# and Value Over Replacement Player
features = ["PER", "BPM", "VORP", "PTS", "AST", "TRB", "TS%",	"WS"]


X_train = df_no_2023[features]
y_train = df_no_2023["Voting Share"]

transformer = make_column_transformer(
    (StandardScaler(), features),
    remainder="passthrough"
)

In [ ]:
pipeline = make_pipeline(
    transformer,
    RandomForestRegressor(random_state=42, n_estimators=100, min_samples_leaf=9, min_samples_split=9, max_features='log2')
)

In [ ]:
X_test = df_2023[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2023["Predicted"] = pipeline.predict(X_test)
df_2023.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
490,31,31,Nikola Jokić,C,27,DEN,69,2323,31.5,0.701,...,1.3,0.7,0.632,0.383,0.822,14.9,0.308,2023,0.674,0.668629
476,17,17,Joel Embiid,C,28,PHI,66,2284,31.4,0.655,...,1.0,1.7,0.548,0.330,0.857,12.3,0.259,2023,0.915,0.526593
473,14,14,Luka Dončić,PG,23,DAL,66,2391,28.7,0.609,...,1.4,0.5,0.496,0.342,0.742,10.2,0.204,2023,0.010,0.401426
459,0,0,Giannis Antetokounmpo,PF,28,MIL,63,2024,29.0,0.605,...,0.8,0.8,0.553,0.275,0.645,8.6,0.204,2023,0.606,0.238831
467,8,8,Jimmy Butler,PF,33,MIA,64,2138,27.6,0.647,...,1.8,0.3,0.539,0.350,0.850,12.3,0.277,2023,0.003,0.231151
480,21,21,Shai Gilgeous-Alexander,PG,24,OKC,68,2416,27.2,0.626,...,1.6,1.0,0.510,0.345,0.905,11.4,0.226,2023,0.046,0.157505
494,35,35,Damian Lillard,PG,32,POR,58,2107,26.7,0.645,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2023,0.000,0.098409
505,46,46,Jayson Tatum,SF,24,BOS,74,2732,23.7,0.607,...,1.1,0.7,0.466,0.350,0.854,10.5,0.185,2023,0.280,0.092832
503,44,44,Domantas Sabonis,C,26,SAC,79,2736,23.5,0.668,...,0.8,0.5,0.615,0.373,0.742,12.6,0.221,2023,0.027,0.071386
470,11,11,Stephen Curry,PG,34,GSW,56,1941,24.1,0.656,...,0.9,0.4,0.493,0.427,0.915,7.8,0.192,2023,0.005,0.070759


## 2022 Test - Jokic Predicted MVP, Jokic Actual

In [ ]:
df_2022 = df[df["Season"] == 2022].fillna(0)
df_no_2022 = df[df["Season"] != 2022].fillna(0)
X_train = df_no_2022[features]
y_train = df_no_2022["Voting Share"]
X_test = df_2022[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2022["Predicted"] = pipeline.predict(X_test)
df_2022.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
437,30,30,Nikola Jokić,C,26,DEN,74,2476,32.8,0.661,...,1.5,0.9,0.583,0.337,0.810,15.2,0.296,2022,0.875,0.743299
408,1,1,Giannis Antetokounmpo,PF,27,MIL,67,2204,32.1,0.633,...,1.1,1.4,0.553,0.293,0.722,12.9,0.281,2022,0.595,0.641860
423,16,16,Joel Embiid,C,27,PHI,68,2297,31.2,0.616,...,1.1,1.5,0.499,0.371,0.814,12.0,0.252,2022,0.706,0.523133
420,13,13,Luka Dončić,PG,22,DAL,65,2301,25.1,0.571,...,1.2,0.6,0.457,0.353,0.744,7.6,0.159,2022,0.146,0.181037
436,29,29,LeBron James,C,37,LAL,56,2084,26.2,0.619,...,1.3,1.1,0.524,0.359,0.756,7.5,0.172,2022,0.001,0.093685
454,47,47,Jayson Tatum,SF,23,BOS,76,2731,21.8,0.578,...,1.0,0.6,0.453,0.353,0.853,9.6,0.169,2022,0.043,0.078049
458,51,51,Trae Young,PG,23,ATL,76,2652,25.4,0.603,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2022,0.000,0.069219
421,14,14,Kevin Durant,PF,33,BRK,55,2047,25.6,0.634,...,0.9,0.9,0.518,0.383,0.910,8.4,0.198,2022,0.001,0.061813
419,12,12,DeMar DeRozan,PF,32,CHI,76,2743,23.1,0.590,...,0.9,0.3,0.504,0.352,0.877,8.8,0.154,2022,0.001,0.054935
455,48,48,Karl-Anthony Towns,C,26,MIN,74,2476,24.1,0.640,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2022,0.000,0.052946


## 2021 Test - Jokic Predicted MVP, Jokic Actual



In [ ]:
df_2021 = df[df["Season"] == 2021].fillna(0)
df_no_2021 = df[df["Season"] != 2021].fillna(0)
X_train = df_no_2021[features]
y_train = df_no_2021["Voting Share"]
X_test = df_2021[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2021["Predicted"] = pipeline.predict(X_test)
df_2021.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
379,25,25,Nikola Jokić,C,25,DEN,72,2488,31.3,0.647,...,1.3,0.7,0.566,0.388,0.868,15.6,0.301,2021,0.961,6.877433e-01
355,1,1,Giannis Antetokounmpo,PF,26,MIL,61,2013,29.2,0.633,...,1.2,1.2,0.569,0.303,0.685,10.2,0.244,2021,0.345,2.961016e-01
364,10,10,Stephen Curry,PG,32,GSW,63,2152,26.3,0.655,...,1.2,0.1,0.482,0.421,0.916,9.0,0.201,2021,0.449,2.213897e-01
383,29,29,Damian Lillard,PG,30,POR,67,2398,25.6,0.623,...,0.9,0.3,0.451,0.391,0.928,10.4,0.209,2021,0.038,1.039405e-01
366,12,12,Luka Dončić,PG,21,DAL,66,2262,25.3,0.587,...,1.0,0.5,0.479,0.350,0.730,7.7,0.163,2021,0.042,7.068042e-02
405,51,51,Zion Williamson,PF,20,NOP,61,2026,27.1,0.649,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2021,0.000,6.819695e-02
358,4,4,Bradley Beal,SG,27,WAS,60,2147,22.7,0.593,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2021,0.000,5.407684e-02
380,26,26,Zach LaVine,SG,25,CHI,58,2034,21.5,0.634,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2021,0.000,4.714084e-02
367,13,13,Kevin Durant,PF,32,BRK,35,1157,26.4,0.666,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2021,0.000,3.795242e-02
377,23,23,Kyrie Irving,PG,28,BRK,54,1886,24.4,0.614,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2021,0.000,3.073427e-02


## 2020 Test - Giannis Predicted MVP, Giannis Actual

In [ ]:
df_2020 = df[df["Season"] == 2020].fillna(0)
df_no_2020 = df[df["Season"] != 2020].fillna(0)
X_train = df_no_2020[features]
y_train = df_no_2020["Voting Share"]
X_test = df_2020[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2020["Predicted"] = pipeline.predict(X_test)
df_2020.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
310,1,1,Giannis Antetokounmpo,PF,25,MIL,63,1917,31.9,0.613,...,1.0,1.0,0.553,0.304,0.633,11.1,0.279,2020,0.952,5.074767e-01
322,13,13,James Harden,SG,30,HOU,68,2483,29.1,0.626,...,1.8,0.9,0.444,0.355,0.865,13.1,0.254,2020,0.363,4.853453e-01
332,23,23,Damian Lillard,PG,29,POR,66,2474,26.9,0.627,...,1.1,0.3,0.463,0.401,0.888,11.6,0.225,2020,0.023,1.935948e-01
330,21,21,Kawhi Leonard,SF,28,LAC,57,1848,26.9,0.589,...,1.8,0.6,0.470,0.378,0.886,8.7,0.226,2020,0.166,1.555158e-01
318,9,9,Luka Dončić,PG,20,DAL,61,2047,27.6,0.585,...,1.0,0.2,0.463,0.316,0.758,8.8,0.207,2020,0.198,1.318393e-01
327,18,18,LeBron James,PG,35,LAL,67,2316,25.5,0.577,...,1.2,0.5,0.493,0.348,0.693,9.8,0.204,2020,0.746,1.193419e-01
311,2,2,Bradley Beal,SG,26,WAS,57,2053,23.2,0.579,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2020,0.000,6.131457e-02
353,44,44,Trae Young,PG,21,ATL,60,2120,23.9,0.595,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2020,0.000,4.722955e-02
326,17,17,Kyrie Irving,PG,27,BRK,20,658,26.2,0.595,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2020,0.000,4.507710e-02
347,38,38,Karl-Anthony Towns,C,24,MIN,35,1187,26.5,0.642,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2020,0.000,3.706373e-02


## 2019 Test - (CONTROVERSIAL MVP) Giannis Predicted MVP, Giannis Actual, Harden close second

In [ ]:
df_2019 = df[df["Season"] == 2019].fillna(0)
df_no_2019 = df[df["Season"] != 2019].fillna(0)
X_train = df_no_2019[features]
y_train = df_no_2019["Voting Share"]
X_test = df_2019[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2019["Predicted"] = pipeline.predict(X_test)
df_2019.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
270,0,0,Giannis Antetokounmpo,PF,24,MIL,72,2358,30.9,0.644,...,1.3,1.5,0.578,0.256,0.729,14.4,0.292,2019,0.932,0.727485
287,17,17,James Harden,PG,29,HOU,78,2867,30.6,0.616,...,2.0,0.7,0.442,0.368,0.879,15.2,0.254,2019,0.768,0.727055
278,8,8,Anthony Davis,C,25,NOP,56,1850,30.3,0.597,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2019,0.000,0.380799
291,21,21,Nikola Jokić,C,23,DEN,80,2504,26.3,0.589,...,1.4,0.7,0.511,0.307,0.821,11.8,0.226,2019,0.210,0.308689
284,14,14,Paul George,SF,28,OKC,77,2841,23.3,0.583,...,2.2,0.4,0.438,0.386,0.839,11.9,0.201,2019,0.352,0.220864
293,23,23,Damian Lillard,PG,28,POR,80,2838,23.7,0.588,...,1.1,0.4,0.444,0.369,0.912,12.1,0.205,2019,0.068,0.122400
277,7,7,Stephen Curry,PG,30,GSW,69,2331,24.4,0.641,...,1.3,0.4,0.472,0.437,0.916,9.7,0.199,2019,0.173,0.092484
281,11,11,Kevin Durant,SF,30,GSW,78,2702,24.2,0.631,...,0.7,1.1,0.521,0.353,0.885,11.5,0.204,2019,0.025,0.089046
303,33,33,Karl-Anthony Towns,C,23,MIN,77,2545,26.3,0.622,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2019,0.000,0.078335
290,20,20,LeBron James,SF,34,LAL,55,1937,25.6,0.588,...,1.3,0.6,0.510,0.339,0.665,7.2,0.179,2019,0.001,0.066755


## 2018 Test - Harden MVP, Harden Actual

In [ ]:
df_2018 = df[df["Season"] == 2018].fillna(0)
df_no_2018 = df[df["Season"] != 2018].fillna(0)
X_train = df_no_2018[features]
y_train = df_no_2018["Voting Share"]
X_test = df_2018[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2018["Predicted"] = pipeline.predict(X_test)
df_2018.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
251,14,14,James Harden,SG,28,HOU,72,2551,29.8,0.619,...,1.8,0.7,0.449,0.367,0.858,15.4,0.289,2018,0.955,0.586479
255,18,18,LeBron James,PF,33,CLE,82,3026,28.6,0.621,...,1.4,0.9,0.542,0.367,0.731,14.0,0.221,2018,0.731,0.482337
257,20,20,Damian Lillard,PG,27,POR,73,2670,25.2,0.594,...,1.1,0.4,0.439,0.361,0.916,12.6,0.227,2018,0.205,0.217812
269,32,32,Russell Westbrook,PG,29,OKC,80,2914,24.7,0.524,...,1.8,0.3,0.449,0.298,0.737,10.1,0.166,2018,0.075,0.188512
237,0,0,Giannis Antetokounmpo,PF,23,MIL,75,2756,27.3,0.598,...,1.5,1.4,0.529,0.307,0.760,11.9,0.207,2018,0.074,0.184082
244,7,7,Stephen Curry,PG,29,GSW,51,1631,28.2,0.675,...,1.6,0.2,0.495,0.423,0.921,9.1,0.267,2018,0.005,0.125525
247,10,10,Kevin Durant,SF,29,GSW,68,2325,26.0,0.640,...,0.7,1.8,0.516,0.419,0.889,10.4,0.215,2018,0.065,0.117171
256,19,19,Nikola Jokić,C,22,DEN,75,2443,24.4,0.603,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2018,0.000,0.050796
264,27,27,Chris Paul,PG,32,HOU,58,1847,24.4,0.604,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2018,0.000,0.026409
263,26,26,Victor Oladipo,SG,25,IND,75,2552,23.1,0.577,...,2.4,0.8,0.477,0.371,0.799,8.2,0.155,2018,0.002,0.026344


## 2017 Test - Russ Predicted, Russ Actual

In [ ]:
df_2017 = df[df["Season"] == 2017].fillna(0)
df_no_2017 = df[df["Season"] != 2017].fillna(0)
X_train = df_no_2017[features]
y_train = df_no_2017["Voting Share"]
X_test = df_2017[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2017["Predicted"] = pipeline.predict(X_test)
df_2017.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
236,28,28,Russell Westbrook,PG,28,OKC,81,2802,30.6,0.554,...,1.6,0.4,0.425,0.343,0.845,13.1,0.224,2017,0.879,0.624256
219,11,11,James Harden,PG,27,HOU,81,2947,27.4,0.613,...,1.5,0.5,0.440,0.347,0.847,15.0,0.245,2017,0.746,0.512426
224,16,16,Kawhi Leonard,SF,25,SAS,74,2474,27.6,0.610,...,1.8,0.7,0.485,0.380,0.880,13.6,0.264,2017,0.495,0.352455
222,14,14,LeBron James,SF,32,CLE,74,2794,27.0,0.619,...,1.2,0.6,0.548,0.363,0.674,12.9,0.221,2017,0.330,0.323561
210,2,2,Jimmy Butler,SF,27,CHI,76,2809,25.1,0.586,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2017,0.000,0.221758
208,0,0,Giannis Antetokounmpo,SF,22,MIL,80,2845,26.1,0.599,...,1.6,1.9,0.521,0.272,0.770,12.4,0.210,2017,0.007,0.221445
216,8,8,Kevin Durant,PF,28,GSW,62,2070,27.6,0.651,...,1.1,1.6,0.537,0.375,0.875,12.0,0.278,2017,0.002,0.207100
213,5,5,Stephen Curry,PG,28,GSW,79,2638,24.6,0.624,...,1.8,0.2,0.468,0.411,0.898,12.6,0.229,2017,0.051,0.152158
229,21,21,Chris Paul,PG,31,LAC,61,1921,26.2,0.614,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2017,0.000,0.142310
212,4,4,DeMarcus Cousins,C,26,TOT,72,2465,25.8,0.562,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2017,0.000,0.128370


## 2016 Test - Curry predicted, Curry actual

In [ ]:
df_2016 = df[df["Season"] == 2016].fillna(0)
df_no_2016 = df[df["Season"] != 2016].fillna(0)
X_train = df_no_2016[features]
y_train = df_no_2016["Voting Share"]
X_test = df_2016[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2016["Predicted"] = pipeline.predict(X_test)
df_2016.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
187,4,4,Stephen Curry,PG,27,GSW,79,2700,31.5,0.669,...,2.1,0.2,0.504,0.454,0.908,17.9,0.318,2016,1.000,0.737029
189,6,6,Kevin Durant,SF,27,OKC,72,2578,28.2,0.634,...,1.0,1.2,0.505,0.387,0.898,14.5,0.270,2016,0.112,0.605219
196,13,13,LeBron James,SF,31,CLE,76,2709,27.5,0.588,...,1.4,0.6,0.520,0.309,0.731,13.6,0.242,2016,0.482,0.357132
194,11,11,James Harden,SG,26,HOU,82,3125,25.3,0.598,...,1.7,0.6,0.439,0.359,0.860,13.3,0.204,2016,0.007,0.285787
207,24,24,Russell Westbrook,PG,27,OKC,80,2750,27.6,0.554,...,2.0,0.3,0.454,0.296,0.812,14.0,0.245,2016,0.371,0.268525
202,19,19,Chris Paul,PG,30,LAC,74,2420,26.2,0.575,...,2.1,0.2,0.462,0.371,0.896,12.7,0.253,2016,0.082,0.130767
199,16,16,Kyle Lowry,PG,29,TOR,77,2851,22.2,0.578,...,2.1,0.4,0.427,0.388,0.811,11.6,0.196,2016,0.005,0.052355
198,15,15,Damian Lillard,PG,25,POR,75,2676,22.2,0.560,...,0.9,0.4,0.419,0.375,0.892,9.2,0.165,2016,0.020,0.027772
188,5,5,DeMar DeRozan,SG,26,TOR,78,2804,21.5,0.550,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2016,0.000,0.011465
192,9,9,Paul George,SF,25,IND,81,2819,20.9,0.557,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2016,0.000,0.010945


## 2015 Curry Predicted, Curry Actual

In [ ]:
df_2015 = df[df["Season"] == 2015].fillna(0)
df_no_2015 = df[df["Season"] != 2015].fillna(0)
X_train = df_no_2015[features]
y_train = df_no_2015["Voting Share"]
X_test = df_2015[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2015["Predicted"] = pipeline.predict(X_test)
df_2015.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
162,3,3,Stephen Curry,PG,26,GSW,80,2613,28.0,0.638,...,2.0,0.2,0.487,0.443,0.914,15.7,0.288,2015,0.922,4.218761e-01
170,11,11,James Harden,SG,25,HOU,81,2981,26.7,0.605,...,1.9,0.7,0.440,0.375,0.868,16.4,0.265,2015,0.720,4.107650e-01
178,19,19,Chris Paul,PG,29,LAC,82,2857,26.0,0.596,...,1.9,0.2,0.485,0.398,0.900,16.1,0.270,2015,0.095,3.021795e-01
182,23,23,Russell Westbrook,PG,26,OKC,67,2302,29.1,0.536,...,2.1,0.2,0.426,0.299,0.835,10.6,0.222,2015,0.271,2.650729e-01
165,6,6,Kevin Durant,SF,26,OKC,27,913,27.6,0.633,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2015,0.000,1.685391e-01
173,14,14,LeBron James,SF,30,CLE,69,2493,25.9,0.577,...,1.6,0.7,0.488,0.354,0.710,10.4,0.199,2015,0.425,1.009314e-01
161,2,2,DeMarcus Cousins,C,24,SAC,59,2013,25.2,0.545,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2015,0.000,2.960431e-02
175,16,16,Damian Lillard,PG,24,POR,82,2925,20.7,0.560,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2015,0.000,1.970699e-02
167,8,8,Marc Gasol,C,30,MEM,81,2687,21.7,0.558,...,0.9,1.6,0.494,0.176,0.795,10.2,0.182,2015,0.002,1.433453e-02
169,10,10,Blake Griffin,PF,25,LAC,67,2356,22.8,0.551,...,0.9,0.5,0.502,0.400,0.728,9.0,0.183,2015,0.002,9.145771e-03


## 2014 Durant Predicted and Actual 9/10



In [ ]:
df_2014 = df[df["Season"] == 2014].fillna(0)
df_no_2014 = df[df["Season"] != 2014].fillna(0)
X_train = df_no_2014[features]
y_train = df_no_2014["Voting Share"]
X_test = df_2014[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2014["Predicted"] = pipeline.predict(X_test)
df_2014.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
139,4,4,Kevin Durant,SF,25,OKC,81,3122,29.8,0.635,...,1.3,0.7,0.503,0.391,0.873,19.2,0.295,2014,0.986,0.627265
147,12,12,LeBron James,PF,29,MIA,77,2902,29.3,0.649,...,1.6,0.3,0.567,0.379,0.750,15.9,0.264,2014,0.713,0.532582
151,16,16,Kevin Love,PF,25,MIN,77,2797,26.9,0.591,...,0.8,0.5,0.457,0.376,0.821,14.3,0.245,2014,0.020,0.424198
137,2,2,Stephen Curry,PG,25,GSW,78,2846,24.1,0.610,...,1.6,0.2,0.471,0.424,0.885,13.4,0.225,2014,0.053,0.250552
154,19,19,Chris Paul,PG,28,LAC,62,2171,25.9,0.580,...,2.5,0.1,0.467,0.368,0.855,12.2,0.270,2014,0.036,0.176655
144,9,9,James Harden,SG,24,HOU,73,2777,23.5,0.618,...,1.6,0.4,0.456,0.366,0.866,12.8,0.221,2014,0.068,0.090366
143,8,8,Blake Griffin,PF,24,LAC,80,2863,23.9,0.583,...,1.2,0.6,0.528,0.273,0.715,12.2,0.205,2014,0.347,0.028186
142,7,7,Paul George,SF,23,IND,80,2898,20.1,0.555,...,1.9,0.3,0.424,0.364,0.864,10.8,0.178,2014,0.026,0.023318
152,17,17,Kyle Lowry,PG,27,TOR,79,2862,20.1,0.567,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2014,0.000,0.018824
158,23,23,Russell Westbrook,PG,25,OKC,46,1412,24.7,0.545,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2014,0.000,0.017321


## 2013 - LeBron Predicted, LeBron MVP 10/11

In [ ]:
df_2013 = df[df["Season"] == 2013].fillna(0)
df_no_2013 = df[df["Season"] != 2013].fillna(0)
X_train = df_no_2013[features]
y_train = df_no_2013["Voting Share"]
X_test = df_2013[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2013["Predicted"] = pipeline.predict(X_test)
df_2013.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
124,9,9,LeBron James,PF,28,MIA,76,2877,31.6,0.640,...,1.7,0.9,0.565,0.406,0.753,19.3,0.322,2013,0.998,0.726854
117,2,2,Kevin Durant,SF,24,OKC,81,3119,28.3,0.647,...,1.4,1.3,0.510,0.416,0.905,18.9,0.291,2013,0.632,0.488643
128,13,13,Chris Paul,PG,27,LAC,70,2335,26.4,0.594,...,2.4,0.1,0.481,0.328,0.885,13.9,0.287,2013,0.239,0.250337
121,6,6,James Harden,SG,23,HOU,78,2985,22.9,0.600,...,1.8,0.5,0.438,0.368,0.851,12.8,0.206,2013,0.027,0.138407
115,0,0,Kobe Bryant,SG,34,LAL,78,3013,23.0,0.570,...,1.4,0.3,0.463,0.324,0.839,10.9,0.174,2013,0.152,0.086912
116,1,1,Stephen Curry,PG,24,GSW,78,2983,21.3,0.589,...,1.6,0.2,0.451,0.453,0.900,11.2,0.180,2013,0.002,0.048860
134,19,19,Russell Westbrook,PG,24,OKC,82,2861,23.9,0.532,...,1.8,0.3,0.438,0.323,0.800,11.6,0.195,2013,0.007,0.033227
131,16,16,Dwyane Wade,SG,31,MIA,69,2391,24.0,0.571,...,1.9,0.8,0.521,0.258,0.725,9.6,0.192,2013,0.004,0.017976
120,5,5,Blake Griffin,PF,23,LAC,80,2598,22.4,0.572,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2013,0.000,0.017231
119,4,4,Paul George,SF,22,IND,79,2972,16.8,0.531,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2013,0.000,0.007705


## 2012 - LeBron Predicted, LeBron Actual 11/12

In [ ]:
df_2012 = df[df["Season"] == 2012].fillna(0)
df_no_2012 = df[df["Season"] != 2012].fillna(0)
X_train = df_no_2012[features]
y_train = df_no_2012["Voting Share"]
X_test = df_2012[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2012["Predicted"] = pipeline.predict(X_test)
df_2012.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
103,9,9,LeBron James,SF,27,MIA,62,2326,30.7,0.605,...,1.9,0.8,0.531,0.362,0.771,14.5,0.298,2012,0.888,6.686216e-01
107,13,13,Chris Paul,PG,26,LAC,60,2181,27.0,0.581,...,2.5,0.1,0.478,0.371,0.861,12.7,0.278,2012,0.318,2.203264e-01
96,2,2,Kevin Durant,SF,23,OKC,66,2546,26.2,0.610,...,1.3,1.2,0.496,0.387,0.860,12.2,0.230,2012,0.735,1.523275e-01
111,17,17,Dwyane Wade,SG,30,MIA,49,1625,26.3,0.559,...,1.7,1.3,0.497,0.268,0.791,7.7,0.227,2012,0.005,2.561948e-02
95,1,1,Kobe Bryant,SG,33,LAL,58,2232,21.9,0.527,...,1.2,0.3,0.430,0.303,0.845,6.2,0.132,2012,0.291,1.953277e-02
100,6,6,James Harden,SG,22,OKC,62,1946,21.1,0.660,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2012,0.000,1.535593e-02
109,15,15,Derrick Rose,PG,23,CHI,39,1375,23.0,0.532,...,0.9,0.7,0.435,0.312,0.812,6.0,0.211,2012,0.004,7.643642e-03
113,19,19,Russell Westbrook,PG,23,OKC,66,2331,22.9,0.538,...,1.7,0.3,0.457,0.316,0.823,7.9,0.163,2012,0.003,4.436198e-03
101,7,7,Kyrie Irving,PG,19,CLE,51,1558,21.4,0.566,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2012,0.000,1.885043e-03
108,14,14,Paul Pierce,SF,34,BOS,61,2075,19.6,0.567,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2012,0.000,1.628895e-03


## 2011 (CONTROVERSIAL) LeBron Predicted, DRose Actual

In [ ]:
df_2011 = df[df["Season"] == 2011].fillna(0)
df_no_2011 = df[df["Season"] != 2011].fillna(0)
X_train = df_no_2011[features]
y_train = df_no_2011["Voting Share"]
X_test = df_2011[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2011["Predicted"] = pipeline.predict(X_test)
df_2011.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
85,8,8,LeBron James,SF,26,MIA,79,3063,27.3,0.594,...,1.6,0.6,0.510,0.330,0.759,15.6,0.244,2011,0.431,0.382293
90,13,13,Dwyane Wade,SG,29,MIA,76,2823,25.6,0.581,...,1.5,1.1,0.500,0.306,0.758,12.8,0.218,2011,0.020,0.174571
89,12,12,Derrick Rose,PG,22,CHI,81,3026,23.5,0.550,...,1.0,0.6,0.445,0.332,0.858,13.1,0.208,2011,0.977,0.152184
88,11,11,Chris Paul,PG,25,NOH,80,2880,23.7,0.578,...,2.4,0.1,0.463,0.388,0.878,13.9,0.232,2011,0.002,0.149825
77,0,0,Kobe Bryant,SG,32,LAL,82,2779,23.9,0.548,...,1.2,0.1,0.451,0.323,0.828,10.3,0.178,2011,0.354,0.043010
92,15,15,Russell Westbrook,PG,22,OKC,82,2847,23.6,0.538,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2011,0.000,0.018659
83,6,6,Blake Griffin,PF,21,LAC,82,3112,21.9,0.549,...,0.8,0.5,0.506,0.292,0.642,9.8,0.152,2011,0.004,0.013626
82,5,5,Manu Ginóbili,SG,33,SAS,80,2426,21.7,0.581,...,1.5,0.4,0.433,0.349,0.871,9.9,0.195,2011,0.017,0.012130
93,16,16,Deron Williams,PG,26,TOT,65,2465,21.1,0.566,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2011,0.000,0.002720
79,2,2,Monta Ellis,SG,25,GSW,80,3227,18.6,0.536,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2011,0.000,0.000275


## 2010 Testing - LeBron Predicted and Actual

In [ ]:
df_2010 = df[df["Season"] == 2010].fillna(0)
df_no_2010 = df[df["Season"] != 2010].fillna(0)
X_train = df_no_2010[features]
y_train = df_no_2010["Voting Share"]
X_test = df_2010[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2010["Predicted"] = pipeline.predict(X_test)
df_2010.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
64,9,9,LeBron James,SF,25,CLE,76,2966,31.1,0.604,...,1.6,1.0,0.503,0.333,0.767,18.5,0.299,2010,0.980,6.956127e-01
74,19,19,Dwyane Wade,SG,28,MIA,77,2792,28.0,0.562,...,1.8,1.1,0.476,0.300,0.761,13.0,0.224,2010,0.097,4.278276e-01
56,1,1,Kobe Bryant,SG,31,LAL,73,2835,21.9,0.545,...,1.5,0.3,0.456,0.329,0.811,9.4,0.160,2010,0.487,3.529218e-02
72,17,17,Josh Smith,PF,24,ATL,81,2871,21.0,0.536,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2010,0.000,2.827723e-02
60,5,5,Pau Gasol,C,29,LAL,65,2403,22.9,0.593,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2010,0.000,2.065456e-02
61,6,6,Manu Ginóbili,SG,32,SAS,75,2150,22.5,0.584,...,1.4,0.3,0.441,0.377,0.870,9.7,0.216,2010,0.002,1.756112e-02
76,21,21,Deron Williams,PG,25,UTA,76,2802,20.6,0.574,...,1.3,0.2,0.469,0.371,0.801,10.3,0.177,2010,0.006,1.533895e-02
67,12,12,David Lee,C,26,NYK,81,3019,22.2,0.584,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2010,0.000,1.518090e-02
68,13,13,Steve Nash,PG,35,PHO,81,2660,21.6,0.615,...,0.5,0.1,0.507,0.426,0.938,9.9,0.178,2010,0.040,1.322004e-02
69,14,14,Chris Paul,PG,24,NOH,45,1712,23.7,0.584,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2010,0.000,1.218044e-02


##2009 Test - LeBron Predicted and Actual

In [ ]:
df_2009 = df[df["Season"] == 2009].fillna(0)
df_no_2009 = df[df["Season"] != 2009].fillna(0)
X_train = df_no_2009[features]
y_train = df_no_2009["Voting Share"]
X_test = df_2009[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2009["Predicted"] = pipeline.predict(X_test)
df_2009.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
42,12,12,LeBron James,SF,24,CLE,81,3054,31.7,0.591,...,1.7,1.1,0.489,0.344,0.780,20.3,0.318,2009,0.969,0.712122
53,23,23,Dwyane Wade,SG,27,MIA,79,3048,30.4,0.574,...,2.2,1.3,0.491,0.317,0.765,14.7,0.232,2009,0.562,0.682130
47,17,17,Chris Paul,PG,23,NOH,78,3002,30.0,0.599,...,2.8,0.1,0.503,0.364,0.868,18.3,0.292,2009,0.159,0.637569
31,1,1,Kobe Bryant,SG,30,LAL,82,2960,24.4,0.561,...,1.5,0.5,0.467,0.351,0.856,12.7,0.206,2009,0.577,0.143291
51,21,21,Brandon Roy,SG,24,POR,78,2903,24.0,0.573,...,1.1,0.3,0.480,0.377,0.824,13.5,0.223,2009,0.006,0.109105
36,6,6,Pau Gasol,C,28,LAL,81,2999,22.2,0.617,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2009,0.000,0.081093
34,4,4,Tim Duncan,C,32,SAS,75,2524,24.4,0.549,...,0.5,1.7,0.504,0.000,0.692,10.1,0.191,2009,0.002,0.022126
48,18,18,Paul Pierce,SF,31,BOS,81,3035,17.7,0.582,...,1.0,0.3,0.457,0.391,0.830,10.3,0.164,2009,0.017,0.011535
54,24,24,Mo Williams,PG,26,CLE,81,2834,17.2,0.588,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2009,0.000,0.006375
37,7,7,Manu Ginóbili,SG,31,SAS,44,1181,22.9,0.594,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2009,0.000,0.004364


##2008 Test - LeBron Predicted, Kobe Actual

In [ ]:
df_2008 = df[df["Season"] == 2008].fillna(0)
df_no_2008 = df[df["Season"] != 2008].fillna(0)
X_train = df_no_2008[features]
y_train = df_no_2008["Voting Share"]
X_test = df_2008[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2008["Predicted"] = pipeline.predict(X_test)
df_2008.sort_values("Predicted", ascending=False)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Age,Tm,G,MP_x,PER,TS%,...,"('Per Game', 'STL')","('Per Game', 'BLK')","('Shooting', 'FG%')","('Shooting', '3P%')","('Shooting', 'FT%')","('Advanced', 'WS')","('Advanced', 'WS/48')",Season,Voting Share,Predicted
13,13,13,LeBron James,SF,23,CLE,75,3027,29.1,0.568,...,1.8,1.1,0.484,0.315,0.712,15.2,0.242,2008,0.0,0.659842
20,20,20,Chris Paul,PG,22,NOH,80,3006,28.3,0.576,...,2.7,0.1,0.488,0.369,0.851,17.8,0.284,2008,0.0,0.587491
2,2,2,Kobe Bryant,SG,29,LAL,82,3192,24.2,0.576,...,1.8,0.5,0.459,0.361,0.840,13.8,0.208,2008,0.0,0.239554
8,8,8,Kevin Garnett,PF,31,BOS,71,2328,25.3,0.588,...,1.4,1.3,0.539,0.000,0.801,12.9,0.265,2008,0.0,0.197280
19,19,19,Dirk Nowitzki,PF,29,DAL,77,2769,24.6,0.585,...,0.7,0.9,0.479,0.359,0.879,12.9,0.223,2008,0.0,0.167618
9,9,9,Manu Ginóbili,SG,30,SAS,74,2299,24.3,0.612,...,1.5,0.4,0.460,0.401,0.860,11.1,0.232,2008,0.0,0.167202
21,21,21,Paul Pierce,SF,30,BOS,80,2874,19.6,0.599,...,1.3,0.5,0.464,0.392,0.843,12.4,0.207,2008,0.0,0.066166
18,18,18,Steve Nash,PG,33,PHO,81,2780,21.1,0.641,...,0.7,0.1,0.504,0.470,0.906,10.5,0.181,2008,0.0,0.019919
0,0,0,Carmelo Anthony,SF,23,DEN,77,2806,21.1,0.568,...,1.3,0.5,0.492,0.354,0.786,8.2,0.140,2008,0.0,0.014548
11,11,11,Andre Iguodala,SF,24,PHI,82,3242,19.0,0.543,...,0.0,0.0,0.000,0.000,0.000,0.0,0.000,2008,0.0,0.007784


81.2%

In [ ]:
diffs = []
dfs = [df_2023, df_2022, df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2014, df_2013, df_2012, df_2011, df_2010, df_2009, df_2008]
for year in dfs:
  max = year.loc[year['Voting Share'].idxmax()]
  print(max['Season'])
  print(max['Voting Share'] - max['Predicted'])
  diffs.append(max['Voting Share'] - max['Predicted'])

2023
0.3884069764848661
2022
0.1317006366871043
2021
0.2732567211330382
2020
0.44452332954480067
2019
0.20451461459954678
2018
0.3685211927837525
2017
0.25474443383716483
2016
0.26297074971878476
2014
0.35873459438421484
2013
0.27114564596534185
2012
0.21937843562981563
2011
0.8248163748360702
2010
0.28438734317010894
2009
0.25687755029653847
2008
-0.014547668157799834


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

diffs = []
dfs = [df_2023, df_2022, df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2014, df_2013, df_2012, df_2011, df_2010, df_2009, df_2008]
for year in dfs:
  max = year.loc[year['Voting Share'].idxmax()]
  print(max['Season'])
  rmse = sqrt(mean_squared_error(year['Voting Share'], year['Predicted']))
  print(rmse)

2023
0.10343578531671772
2022
0.047712024435137265
2021
0.05616706258038118
2020
0.12022698729241392
2019
0.0794136987041736
2018
0.0862624482125235
2017
0.10819774150314238
2016
0.12983687979088554
2014
0.14235373697878984
2013
0.07559694613929975
2012
0.1498496963980447
2011
0.2203774801541358
2010
0.13435267936189788
2009
0.14356477332453108
2008
0.1768213431529452


In [ ]:
!pip install shap

from sklearn.model_selection import train_test_split
import shap

shap.initjs()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
X_train = df[features]
y_train = df["Voting Share"].fillna(0)
X_test = pd.read_csv("2024 (1).csv")[features].fillna(0)



transformer = make_column_transformer(
    (StandardScaler(), features),
    remainder="passthrough"
)

X_train = transformer.fit_transform(X_train)

X_test = transformer.fit_transform(X_test)

model = RandomForestRegressor(random_state=42, n_estimators=100, min_samples_leaf=9, min_samples_split=9, max_features='log2')

model.fit(X_train, y_train)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

FileNotFoundError: [Errno 2] No such file or directory: '2024 (1).csv'

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
shap.summary_plot(shap_values, X_test, feature_names=features, plot_size=(12,8))




In [ ]:
import pandas as pd
import plotly.graph_objects as go

data = {'Year': ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
        'Predicted': ['LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic'],
        'Actual': ['Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Joel Embiid', 'TBD']}
df = pd.DataFrame(data)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='gold',
                align='left'),
    cells=dict(values=[df['Year'], df['Predicted'], df['Actual']],
               fill_color='white',
               align='left'))
])

# Update layout for better display (optional)
fig.update_layout(title='NBA MVP Predicted vs Actual')

# Show the Plotly table
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

data = {'Year': ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
        'Predicted': ['LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic'],
        'Actual': ['Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Joel Embiid', 'TBD']}
df = pd.DataFrame(data)

# Define a list to store cell colors
cell_colors = []

# Define a condition for row coloring
condition_rows = df['Actual'] == df['Predicted']

# Loop through each row to assign colors
for idx, row in df.iterrows():
    if condition_rows[idx]:
        cell_colors.append(['green' for _ in row])
    else:
        cell_colors.append(['red' for _ in row])

# Create the Plotly table with customized cell colors
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='gold',
                align='left'),
    cells=dict(values=[df['Year'], df['Predicted'], df['Actual']],
               fill_color=cell_colors,  # Use the customized cell colors
               align='left'))
])

# Update layout for better display (optional)
fig.update_layout(title='Sample Table')

# Show the Plotly table
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

data = {'Year': ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
        'Predicted': ['LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic'],
        'Actual': ['Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Joel Embiid', 'TBD']}
df = pd.DataFrame(data)

# Define a list to store row colors
row_colors = []

# Loop through each row to assign colors based on condition
for idx, row in df.iterrows():
    if row['Predicted'] == row['Actual']:
        row_colors.append('green')
    else:
        row_colors.append('red')

# Create the Plotly table with customized row colors
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['Year'], df['Predicted'], df['Actual']],
               fill_color='white',
               align='left'),
    row_colors=row_colors  # Use the customized row colors
)])

# Update layout for better display (optional)
fig.update_layout(title='Sample Table')

# Show the Plotly table
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

data = {'Year': ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
        'Predicted': ['LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic'],
        'Actual': ['Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Joel Embiid', 'TBD']}
df = pd.DataFrame(data)

# Define a list to store cell colors
cell_colors = []

# Loop through each row to assign colors based on condition
for idx, row in df.iterrows():
    if row['Actual'] == row['Predicted']:
        cell_colors.append(['green' for _ in row])
    else:
        cell_colors.append(['red' for _ in row])

# Create the Plotly table with customized row colors
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['Year'], df['Predicted'], df['Actual']],
               fill_color=cell_colors,  # Use the customized row colors
               align='left'))
])

# Update layout for better display (optional)
fig.update_layout(title='Sample Table')

# Show the Plotly table
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

data = {
    'Year': ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
    'Predicted': ['LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic'],
    'Actual': ['Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Joel Embiid', 'TBD']
}
df = pd.DataFrame(data)

# Define a list to store row colors
row_colors = []

col_widths = [25, 50, 50]  # Adjust the values as needed

# Loop through each row to assign colors based on condition
for idx, row in df.iterrows():
    if row['Actual'] == row['Predicted']:
        row_colors.append('lightgreen')
    else:
        row_colors.append('lightpink')

# Create the Plotly table with customized row colors
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['Year'], df['Predicted'], df['Actual']],
               fill_color=[row_colors, row_colors, row_colors],  # Use the customized row colors for all columns
               align='left'),
    columnwidth=col_widths)
])

# Update layout for better display (optional)
fig.update_layout(title='MVP Predictor 81.3% accuracy')

# Show the Plotly table
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

data = {
    'Year': ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
    'Predicted': ['LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic', 'Nikola Jokic'],
    'Actual': ['Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokic', 'Nikola Jokic', 'Joel Embiid', 'TBD']
}
df = pd.DataFrame(data)

# Define custom column width
col_widths = [50, 100, 100]  # Adjust the values as needed

# Create the Plotly table with custom column widths
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[df['Year'], df['Predicted'], df['Actual']],
               fill_color='white',
               align='left'),
    columnwidth=col_widths  # Set custom column widths
)])

# Update layout for better display (optional)
fig.update_layout(title='Sample Table')

# Show the Plotly table
fig.show()


In [ ]:
shap.dependence_plot(df.iloc[df["features"]], shap_values, X_test, feature_names=features)


In [ ]:
instance_index = 0
shap.waterfall_plot(explainer.expected_value, shap_values[instance_index], feature_names=features)

In [ ]:
import plotly.graph_objects as go

# Sample data
x_values = [1, 2, 3, 4, 5]
y_values = [10, 15, 13, 18, 20]

# Create a trace for the line plot
trace = go.Scatter(x=x_values, y=y_values, mode='lines+markers')

# Create the figure and add the trace
fig = go.Figure(trace)

# Update layout (optional)
fig.update_layout(title='Line Graph', xaxis_title='X Axis', yaxis_title='Y Axis')

# Show the plot
fig.show()


## 2024 Test

In [ ]:
df_2024 = pd.read_csv("/content/2024 (1).csv")
pd.set_option('display.float_format', lambda x: '%.5f' % x)
X_train = df[features]
y_train = df["Voting Share"].fillna(0)
X_test = df_2024[features].fillna(0.0)
pipeline.fit(X=X_train, y=y_train)
df_2024["Predicted"] = pipeline.predict(X_test)
df_2024["Predicted"] = df_2024["Predicted"]
df_2024[["Player", "Predicted"]].sort_values("Predicted", ascending=False).head(5)

In [ ]:
diffs = []
dfs = [df_2023, df_2022, df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2014, df_2013, df_2012, df_2011, df_2010, df_2009, df_2008]
for year in dfs:
  max = year.loc[year['Voting Share'].idxmax()]
  print(max['Season'], print['Player'])
  print(abs(max['Voting Share'] - max['Predicted']))
  diffs.append(abs(max['Voting Share'] - max['Predicted']))